In [ ]:
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue import DynamicFrame

def sparkUnion(glueContext, unionType, mapping, transformation_ctx) -> DynamicFrame:
    for alias, frame in mapping.items():
        frame.toDF().createOrReplaceTempView(alias)
    result = spark.sql("(select * from source1) UNION " + unionType + " (select * from source2)")
    return DynamicFrame.fromDF(result, glueContext, transformation_ctx)
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Script generated for node Amazon S3
AmazonS3_node1711754192967 = glueContext.create_dynamic_frame.from_options(format_options={}, connection_type="s3", format="parquet", connection_options={"paths": ["s3://de-news-parquet-cleaned/run-1711753965891-part-block-0-r-00000-snappy.parquet"], "recurse": True}, transformation_ctx="AmazonS3_node1711754192967")

# Script generated for node Amazon S3
AmazonS3_node1711754282666 = glueContext.create_dynamic_frame.from_options(format_options={}, connection_type="s3", format="parquet", connection_options={"paths": ["s3://de-news-parquet-cleaned/run-1711754076059-part-block-0-r-00000-snappy.parquet"], "recurse": True}, transformation_ctx="AmazonS3_node1711754282666")

# Script generated for node Amazon S3
AmazonS3_node1711754190843 = glueContext.create_dynamic_frame.from_options(format_options={}, connection_type="s3", format="parquet", connection_options={"paths": ["s3://de-news-parquet-cleaned/run-1711753393084-part-block-0-r-00000-snappy.parquet"], "recurse": True}, transformation_ctx="AmazonS3_node1711754190843")

# Script generated for node Schema Matcher for Union
SchemaMatcherforUnion_node1711754248262 = ApplyMapping.apply(frame=AmazonS3_node1711754192967, mappings=[("author", "string", "author", "string"), ("title", "string", "title", "string"), ("description", "string", "description", "string"), ("source", "string", "source", "string"), ("category", "string", "category", "string"), ("date", "string", "date", "string")], transformation_ctx="SchemaMatcherforUnion_node1711754248262")

# Script generated for node Change Schema
ChangeSchema_node1711754326806 = ApplyMapping.apply(frame=AmazonS3_node1711754282666, mappings=[("author", "string", "author", "string"), ("title", "string", "title", "string"), ("description", "string", "description", "string"), ("source", "string", "source", "string"), ("category", "string", "category", "string"), ("date", "string", "date", "string")], transformation_ctx="ChangeSchema_node1711754326806")

# Script generated for node Union
Union_node1711754635416 = sparkUnion(glueContext, unionType = "ALL", mapping = {"source1": ChangeSchema_node1711754326806, "source2": SchemaMatcherforUnion_node1711754248262}, transformation_ctx = "Union_node1711754635416")

# Script generated for node Union
Union_node1711754704228 = sparkUnion(glueContext, unionType = "ALL", mapping = {"source1": AmazonS3_node1711754190843, "source2": Union_node1711754635416}, transformation_ctx = "Union_node1711754704228")

# Script generated for node Amazon S3
AmazonS3_node1711754716481 = glueContext.write_dynamic_frame.from_options(frame=Union_node1711754704228, connection_type="s3", format="glueparquet", connection_options={"path": "s3://de-news-merge-parquets", "partitionKeys": []}, format_options={"compression": "snappy"}, transformation_ctx="AmazonS3_node1711754716481")

job.commit()